In [ ]:
# prepare data
from utils.util import conll_iob
from utils.util import get_reader, train_model, create_model, save_model, parse_args, get_tagset
train_file = "./training_data/EN-English/en_train.conll"
dev_file = "./training_data/EN-English/en_dev.conll"
encoder_model = "roberta-base"
train_reader = get_reader(file_path=train_file, target_vocab=get_tagset(conll_iob), encoder_model=encoder_model)
dev_reader = get_reader(file_path=dev_file, target_vocab=get_tagset(conll_iob), encoder_model=encoder_model)


Error: Session cannot generate requests

In [5]:
print(train_reader.sentences.__len__())
print(dev_reader.__len__())
from typing import List
def write_roberta_pretrain_sentences(data: List[List], filename: str):
    with open(filename, "w") as f:
        for sentences in data:
            for sentence in sentences:
                f.write("<s> " + sentence + " </s>")
                f.write("\n")

write_roberta_pretrain_sentences([train_reader.sentences, dev_reader.sentences], "roberta.txt")
    

15300
800


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import os

encoder_model = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(encoder_model)
if os.path.exists("./roberta-retrained"):
    model = RobertaForMaskedLM.from_pretrained("./roberta-retrained")
else:
    model = RobertaForMaskedLM.from_pretrained(encoder_model)
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="roberta.txt",
    block_size=512,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
trainer.train()

Error: Session cannot generate requests